In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [2]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [ ]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

# GSAGE

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [4]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

In [5]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.7830, Train: 77.01%, Valid: 54.24% Test: 51.35%
Run: 01, Epoch: 02, Loss: 0.9958, Train: 78.16%, Valid: 57.63% Test: 59.46%
Run: 01, Epoch: 03, Loss: 0.7884, Train: 80.46%, Valid: 55.93% Test: 59.46%
Run: 01, Epoch: 04, Loss: 0.7604, Train: 88.51%, Valid: 59.32% Test: 62.16%
Run: 01, Epoch: 05, Loss: 0.6071, Train: 87.36%, Valid: 59.32% Test: 64.86%
Run: 01, Epoch: 06, Loss: 0.5397, Train: 93.10%, Valid: 61.02% Test: 64.86%
Run: 01, Epoch: 07, Loss: 0.4847, Train: 94.25%, Valid: 61.02% Test: 67.57%
Run: 01, Epoch: 08, Loss: 0.5274, Train: 94.25%, Valid: 62.71% Test: 70.27%
Run: 01, Epoch: 09, Loss: 0.4682, Train: 94.25%, Valid: 64.41% Test: 62.16%
Run: 01, Epoch: 10, Loss: 0.3679, Train: 94.25%, Valid: 64.41% Test: 62.16%
Run: 01, Epoch: 11, Loss: 0.4389, Train: 95.40%, Valid: 61.02% Test: 59.46%
Run: 01, Epoch: 12, Loss: 0.3415, Train: 96.55%, Valid: 62.71% Test: 59.46%
Run: 01, Epoch: 13, Loss: 0.3268, Train: 97.70%, Valid: 62.71% Test: 56.76%
Run: 01, Epo

Run: 02, Epoch: 36, Loss: 0.1134, Train: 100.00%, Valid: 62.71% Test: 59.46%
Run: 02, Epoch: 37, Loss: 0.0755, Train: 100.00%, Valid: 62.71% Test: 59.46%
Run: 02, Epoch: 38, Loss: 0.0879, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 39, Loss: 0.0399, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 40, Loss: 0.0492, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 41, Loss: 0.0539, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 42, Loss: 0.0564, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 43, Loss: 0.0694, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 44, Loss: 0.0540, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 45, Loss: 0.0610, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 46, Loss: 0.0440, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 47, Loss: 0.0604, Train: 100.00%, Valid: 62.71% Test: 56.76%
Run: 02, Epoch: 48, Loss: 0.0818, Train: 100.00%, Valid: 62.71% Test: 56.76%

Run: 03, Epoch: 82, Loss: 0.0498, Train: 100.00%, Valid: 67.80% Test: 67.57%
Run: 03, Epoch: 83, Loss: 0.0505, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 03, Epoch: 84, Loss: 0.0281, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 03, Epoch: 85, Loss: 0.0318, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 03, Epoch: 86, Loss: 0.0127, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 03, Epoch: 87, Loss: 0.0246, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 03, Epoch: 88, Loss: 0.0141, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 03, Epoch: 89, Loss: 0.0274, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 03, Epoch: 90, Loss: 0.0159, Train: 100.00%, Valid: 67.80% Test: 64.86%
Run: 03, Epoch: 91, Loss: 0.0087, Train: 100.00%, Valid: 67.80% Test: 64.86%
Run: 03, Epoch: 92, Loss: 0.0368, Train: 100.00%, Valid: 67.80% Test: 64.86%
Run: 03, Epoch: 93, Loss: 0.0031, Train: 100.00%, Valid: 67.80% Test: 67.57%
Run: 03, Epoch: 94, Loss: 0.0553, Train: 100.00%, Valid: 69.49% Test: 67.57%

Run: 05, Epoch: 26, Loss: 0.1187, Train: 100.00%, Valid: 54.24% Test: 62.16%
Run: 05, Epoch: 27, Loss: 0.0851, Train: 100.00%, Valid: 54.24% Test: 67.57%
Run: 05, Epoch: 28, Loss: 0.0717, Train: 100.00%, Valid: 54.24% Test: 64.86%
Run: 05, Epoch: 29, Loss: 0.0943, Train: 100.00%, Valid: 50.85% Test: 64.86%
Run: 05, Epoch: 30, Loss: 0.1080, Train: 100.00%, Valid: 52.54% Test: 64.86%
Run: 05, Epoch: 31, Loss: 0.0684, Train: 100.00%, Valid: 54.24% Test: 64.86%
Run: 05, Epoch: 32, Loss: 0.0631, Train: 100.00%, Valid: 54.24% Test: 64.86%
Run: 05, Epoch: 33, Loss: 0.0658, Train: 100.00%, Valid: 54.24% Test: 62.16%
Run: 05, Epoch: 34, Loss: 0.0624, Train: 100.00%, Valid: 54.24% Test: 62.16%
Run: 05, Epoch: 35, Loss: 0.0995, Train: 100.00%, Valid: 54.24% Test: 62.16%
Run: 05, Epoch: 36, Loss: 0.0484, Train: 100.00%, Valid: 54.24% Test: 62.16%
Run: 05, Epoch: 37, Loss: 0.0685, Train: 100.00%, Valid: 52.54% Test: 62.16%
Run: 05, Epoch: 38, Loss: 0.0495, Train: 100.00%, Valid: 50.85% Test: 62.16%

Run: 06, Epoch: 75, Loss: 0.0233, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 76, Loss: 0.0082, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 77, Loss: 0.0312, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 78, Loss: 0.0363, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 79, Loss: 0.0441, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 80, Loss: 0.0466, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 81, Loss: 0.0236, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 82, Loss: 0.0275, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 83, Loss: 0.0275, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 84, Loss: 0.0183, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 85, Loss: 0.0340, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 86, Loss: 0.0184, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 06, Epoch: 87, Loss: 0.0209, Train: 100.00%, Valid: 66.10% Test: 59.46%

Run: 08, Epoch: 22, Loss: 0.1418, Train: 98.85%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 23, Loss: 0.1874, Train: 98.85%, Valid: 67.80% Test: 62.16%
Run: 08, Epoch: 24, Loss: 0.1248, Train: 98.85%, Valid: 67.80% Test: 62.16%
Run: 08, Epoch: 25, Loss: 0.1521, Train: 100.00%, Valid: 67.80% Test: 62.16%
Run: 08, Epoch: 26, Loss: 0.1791, Train: 100.00%, Valid: 67.80% Test: 62.16%
Run: 08, Epoch: 27, Loss: 0.1305, Train: 100.00%, Valid: 67.80% Test: 62.16%
Run: 08, Epoch: 28, Loss: 0.1193, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 29, Loss: 0.1352, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 30, Loss: 0.0933, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 31, Loss: 0.1374, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 32, Loss: 0.1109, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 08, Epoch: 33, Loss: 0.0727, Train: 100.00%, Valid: 66.10% Test: 59.46%
Run: 08, Epoch: 34, Loss: 0.0847, Train: 100.00%, Valid: 64.41% Test: 62.16%
Ru

Run: 09, Epoch: 56, Loss: 0.0589, Train: 100.00%, Valid: 72.88% Test: 62.16%
Run: 09, Epoch: 57, Loss: 0.0675, Train: 100.00%, Valid: 72.88% Test: 62.16%
Run: 09, Epoch: 58, Loss: 0.0224, Train: 100.00%, Valid: 74.58% Test: 62.16%
Run: 09, Epoch: 59, Loss: 0.0211, Train: 100.00%, Valid: 74.58% Test: 62.16%
Run: 09, Epoch: 60, Loss: 0.0582, Train: 100.00%, Valid: 76.27% Test: 62.16%
Run: 09, Epoch: 61, Loss: 0.0511, Train: 100.00%, Valid: 76.27% Test: 62.16%
Run: 09, Epoch: 62, Loss: 0.0395, Train: 100.00%, Valid: 76.27% Test: 62.16%
Run: 09, Epoch: 63, Loss: 0.0489, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 09, Epoch: 64, Loss: 0.0143, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 09, Epoch: 65, Loss: 0.0198, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 09, Epoch: 66, Loss: 0.0599, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 09, Epoch: 67, Loss: 0.0267, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 09, Epoch: 68, Loss: 0.0452, Train: 100.00%, Valid: 76.27% Test: 64.86%

# WISE EMBEDDING

In [6]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [7]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [8]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [9]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [10]:
print(Fec)

[[3, 2, 3, 4, 4], [51, 44, 51, 54, 39], [35, 24, 32, 32, 24], [59, 47, 60, 70, 39], [53, 56, 63, 71, 55], [115, 93, 122, 132, 92], [129, 99, 131, 136, 156], [116, 161, 151, 147, 122], [136, 138, 175, 157, 145], [18, 17, 18, 18, 18], [42, 38, 45, 43, 48], [63, 57, 71, 86, 55], [54, 65, 64, 64, 69], [63, 59, 70, 82, 55], [43, 42, 45, 46, 36], [51, 47, 50, 62, 47], [97, 96, 129, 116, 85], [81, 95, 85, 87, 72], [34, 33, 36, 36, 33], [211, 159, 208, 206, 263], [22, 23, 21, 23, 24], [194, 175, 257, 222, 179], [165, 103, 139, 137, 97], [53, 49, 54, 58, 52], [61, 85, 79, 80, 73], [37, 28, 33, 33, 28], [36, 40, 40, 48, 29], [109, 93, 128, 114, 84], [131, 171, 155, 160, 131], [108, 147, 133, 129, 111], [105, 85, 125, 134, 91], [46, 31, 40, 40, 32], [203, 200, 276, 241, 173], [210, 133, 168, 177, 122], [54, 45, 62, 68, 48], [54, 41, 49, 51, 36], [32, 33, 31, 34, 28], [114, 104, 141, 122, 98], [41, 48, 45, 48, 41], [41, 41, 42, 50, 35], [42, 42, 45, 47, 34], [88, 70, 93, 99, 78], [108, 72, 91, 96,

In [11]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [12]:
print(SFec)

[[3, 2, 2, 2, 4], [40, 44, 42, 39, 39], [29, 24, 21, 15, 24], [42, 47, 49, 40, 39], [36, 56, 50, 41, 55], [85, 93, 94, 58, 92], [81, 99, 90, 47, 156], [75, 161, 114, 50, 122], [86, 138, 133, 64, 145], [16, 17, 17, 16, 18], [35, 38, 36, 27, 48], [43, 57, 50, 51, 55], [35, 65, 55, 20, 69], [46, 59, 55, 57, 55], [37, 42, 41, 37, 36], [38, 47, 42, 37, 47], [63, 96, 107, 54, 85], [60, 95, 70, 44, 72], [30, 33, 35, 34, 33], [123, 159, 140, 67, 263], [17, 23, 20, 19, 24], [115, 175, 190, 75, 179], [121, 103, 103, 55, 97], [38, 49, 45, 45, 52], [46, 85, 62, 47, 73], [34, 28, 27, 24, 28], [24, 40, 37, 32, 29], [70, 93, 105, 49, 84], [82, 171, 121, 66, 131], [71, 147, 111, 59, 111], [70, 85, 85, 79, 91], [43, 31, 32, 28, 32], [126, 200, 220, 72, 173], [149, 133, 107, 71, 122], [43, 45, 46, 42, 48], [50, 41, 39, 35, 36], [28, 33, 30, 31, 28], [62, 104, 113, 44, 98], [35, 48, 40, 37, 41], [33, 41, 34, 42, 35], [37, 42, 42, 35, 34], [70, 70, 72, 50, 78], [78, 72, 72, 45, 79], [64, 95, 94, 57, 93], 

In [13]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[  3.,   2.,   3.,  ...,   2.,   2.,   4.],
        [ 51.,  44.,  51.,  ...,  42.,  39.,  39.],
        [ 35.,  24.,  32.,  ...,  21.,  15.,  24.],
        ...,
        [ 69.,  61.,  79.,  ...,  63.,  55.,  59.],
        [ 88.,  85., 101.,  ...,  90.,  52.,  68.],
        [168., 144., 232.,  ..., 157.,  68., 115.]])


'torch.FloatTensor'

# W-GSAGE

In [14]:
data.x=CC_domain
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [16]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.7838, Train: 19.54%, Valid: 20.34% Test: 24.32%
Run: 01, Epoch: 02, Loss: 1.5707, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 03, Loss: 1.3747, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 04, Loss: 1.3540, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 05, Loss: 1.3624, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 06, Loss: 1.2908, Train: 20.69%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 07, Loss: 1.1782, Train: 21.84%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 08, Loss: 1.1346, Train: 22.99%, Valid: 18.64% Test: 29.73%
Run: 01, Epoch: 09, Loss: 1.1935, Train: 26.44%, Valid: 22.03% Test: 29.73%
Run: 01, Epoch: 10, Loss: 1.1813, Train: 33.33%, Valid: 28.81% Test: 35.14%
Run: 01, Epoch: 11, Loss: 1.0403, Train: 39.08%, Valid: 40.68% Test: 48.65%
Run: 01, Epoch: 12, Loss: 1.0716, Train: 45.98%, Valid: 47.46% Test: 54.05%
Run: 01, Epoch: 13, Loss: 0.9638, Train: 54.02%, Valid: 52.54% Test: 64.86%
Run: 01, Epo

Run: 02, Epoch: 84, Loss: 0.2619, Train: 89.66%, Valid: 79.66% Test: 72.97%
Run: 02, Epoch: 85, Loss: 0.2222, Train: 91.95%, Valid: 77.97% Test: 72.97%
Run: 02, Epoch: 86, Loss: 0.1877, Train: 93.10%, Valid: 77.97% Test: 72.97%
Run: 02, Epoch: 87, Loss: 0.2560, Train: 95.40%, Valid: 77.97% Test: 78.38%
Run: 02, Epoch: 88, Loss: 0.2074, Train: 95.40%, Valid: 77.97% Test: 75.68%
Run: 02, Epoch: 89, Loss: 0.2143, Train: 96.55%, Valid: 79.66% Test: 75.68%
Run: 02, Epoch: 90, Loss: 0.1904, Train: 94.25%, Valid: 81.36% Test: 81.08%
Run: 02, Epoch: 91, Loss: 0.1980, Train: 93.10%, Valid: 81.36% Test: 78.38%
Run: 02, Epoch: 92, Loss: 0.1890, Train: 91.95%, Valid: 84.75% Test: 78.38%
Run: 02, Epoch: 93, Loss: 0.2529, Train: 90.80%, Valid: 86.44% Test: 78.38%
Run: 02, Epoch: 94, Loss: 0.2714, Train: 90.80%, Valid: 81.36% Test: 81.08%
Run: 02, Epoch: 95, Loss: 0.2702, Train: 94.25%, Valid: 79.66% Test: 83.78%
Run: 02, Epoch: 96, Loss: 0.2160, Train: 95.40%, Valid: 76.27% Test: 81.08%
Run: 02, Epo

Run: 04, Epoch: 92, Loss: 0.2038, Train: 94.25%, Valid: 81.36% Test: 75.68%
Run: 04, Epoch: 93, Loss: 0.2161, Train: 94.25%, Valid: 81.36% Test: 75.68%
Run: 04, Epoch: 94, Loss: 0.1763, Train: 94.25%, Valid: 81.36% Test: 75.68%
Run: 04, Epoch: 95, Loss: 0.1739, Train: 95.40%, Valid: 83.05% Test: 78.38%
Run: 04, Epoch: 96, Loss: 0.2058, Train: 95.40%, Valid: 84.75% Test: 81.08%
Run: 04, Epoch: 97, Loss: 0.2774, Train: 95.40%, Valid: 83.05% Test: 83.78%
Run: 04, Epoch: 98, Loss: 0.1534, Train: 96.55%, Valid: 83.05% Test: 83.78%
Run: 04, Epoch: 99, Loss: 0.1522, Train: 96.55%, Valid: 81.36% Test: 83.78%
Run: 04, Epoch: 100, Loss: 0.1474, Train: 97.70%, Valid: 84.75% Test: 81.08%
Run 04:
Highest Train: 97.70
Highest Valid: 86.44
  Final Train: 94.25
   Final Test: 81.08
Run: 05, Epoch: 01, Loss: 1.8041, Train: 18.39%, Valid: 11.86% Test: 13.51%
Run: 05, Epoch: 02, Loss: 1.5532, Train: 27.59%, Valid: 25.42% Test: 45.95%
Run: 05, Epoch: 03, Loss: 1.3647, Train: 43.68%, Valid: 35.59% Test: 43

Run: 07, Epoch: 04, Loss: 1.3173, Train: 63.22%, Valid: 55.93% Test: 51.35%
Run: 07, Epoch: 05, Loss: 1.2988, Train: 58.62%, Valid: 55.93% Test: 37.84%
Run: 07, Epoch: 06, Loss: 1.1460, Train: 57.47%, Valid: 50.85% Test: 37.84%
Run: 07, Epoch: 07, Loss: 1.2084, Train: 56.32%, Valid: 50.85% Test: 40.54%
Run: 07, Epoch: 08, Loss: 1.1614, Train: 57.47%, Valid: 50.85% Test: 43.24%
Run: 07, Epoch: 09, Loss: 1.1479, Train: 59.77%, Valid: 54.24% Test: 48.65%
Run: 07, Epoch: 10, Loss: 1.0828, Train: 62.07%, Valid: 55.93% Test: 45.95%
Run: 07, Epoch: 11, Loss: 1.2067, Train: 63.22%, Valid: 55.93% Test: 51.35%
Run: 07, Epoch: 12, Loss: 1.0803, Train: 65.52%, Valid: 55.93% Test: 51.35%
Run: 07, Epoch: 13, Loss: 0.9927, Train: 66.67%, Valid: 55.93% Test: 51.35%
Run: 07, Epoch: 14, Loss: 0.9304, Train: 67.82%, Valid: 54.24% Test: 56.76%
Run: 07, Epoch: 15, Loss: 0.8931, Train: 68.97%, Valid: 61.02% Test: 54.05%
Run: 07, Epoch: 16, Loss: 0.8729, Train: 72.41%, Valid: 64.41% Test: 59.46%
Run: 07, Epo

Run: 09, Epoch: 16, Loss: 0.8759, Train: 72.41%, Valid: 67.80% Test: 62.16%
Run: 09, Epoch: 17, Loss: 0.8737, Train: 73.56%, Valid: 72.88% Test: 64.86%
Run: 09, Epoch: 18, Loss: 0.9334, Train: 72.41%, Valid: 72.88% Test: 62.16%
Run: 09, Epoch: 19, Loss: 0.8430, Train: 72.41%, Valid: 72.88% Test: 59.46%
Run: 09, Epoch: 20, Loss: 0.7526, Train: 71.26%, Valid: 74.58% Test: 62.16%
Run: 09, Epoch: 21, Loss: 0.7388, Train: 70.11%, Valid: 76.27% Test: 51.35%
Run: 09, Epoch: 22, Loss: 0.6544, Train: 73.56%, Valid: 76.27% Test: 54.05%
Run: 09, Epoch: 23, Loss: 0.7611, Train: 73.56%, Valid: 76.27% Test: 56.76%
Run: 09, Epoch: 24, Loss: 0.7581, Train: 74.71%, Valid: 76.27% Test: 59.46%
Run: 09, Epoch: 25, Loss: 0.7321, Train: 77.01%, Valid: 76.27% Test: 62.16%
Run: 09, Epoch: 26, Loss: 0.6259, Train: 77.01%, Valid: 74.58% Test: 62.16%
Run: 09, Epoch: 27, Loss: 0.6969, Train: 77.01%, Valid: 72.88% Test: 62.16%
Run: 09, Epoch: 28, Loss: 0.6979, Train: 77.01%, Valid: 72.88% Test: 59.46%
Run: 09, Epo

# Topological Encoding 

In [4]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell')
data = dataset[0]
print(data)

Data(x=[183, 1703], edge_index=[2, 298], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])


In [5]:
print(data.edge_index.numpy())

[[  0   0   1   2   2   6   6   8   8   8   8   8   9  10  10  10  10  12
   13  18  19  20  20  20  20  20  21  21  21  21  22  22  24  24  24  24
   24  24  24  25  25  27  27  27  28  28  29  31  33  35  41  41  42  43
   48  48  52  53  53  57  57  57  57  57  57  57  57  57  57  57  57  57
   57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57
   57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57
   57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57
   57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57  57
   57  57  57  57  57  57  57  57  62  62  63  64  65  65  66  66  66  66
   66  66  66  66  66  67  67  67  67  67  67  67  67  69  70  70  72  72
   73  74  78  78  82  83  83  83  83  83  83  83  85  87  89  91  94  95
   96  96  96  97  97  97  97  97  99 101 101 101 101 102 104 104 105 109
  109 109 110 110 115 116 117 120 120 120 121 122 122 122 125 125 126 128
  129 133 133 133 134 135 135 135 137 

In [7]:
Number_nodes=len(data.y)
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [8]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [9]:
print(G.number_of_edges())

298


In [10]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [11]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [12]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  69 

2  |  40 

3  |  22 

4  |  20 

5  |  13 

6  |  4 

7  |  5 

8  |  3 

9  |  3 

10  |  2 

12  |  1 

94  |  1 



In [13]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,50]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 182 (100%)

In [26]:
print( Node_Edge)

[[7, 19], [99, 131], [6, 9], [111, 163], [18, 29], [109, 141], [12, 26], [96, 120], [18, 41], [5, 4], [11, 16], [95, 119], [3, 2], [16, 23], [95, 119], [95, 119], [95, 119], [95, 119], [98, 126], [4, 3], [16, 36], [16, 22], [10, 13], [9, 10], [104, 138], [8, 11], [97, 121], [11, 16], [22, 42], [96, 121], [108, 153], [98, 126], [95, 119], [96, 120], [95, 119], [3, 2], [3, 2], [95, 119], [12, 19], [12, 20], [95, 119], [111, 158], [97, 124], [97, 121], [5, 4], [95, 119], [100, 128], [15, 27], [6, 5], [19, 31], [101, 130], [3, 2], [5, 5], [101, 129], [5, 7], [95, 119], [5, 5], [152, 246], [95, 119], [95, 119], [95, 119], [95, 119], [7, 6], [13, 20], [5, 6], [12, 25], [22, 41], [21, 34], [100, 127], [16, 23], [6, 9], [95, 119], [97, 124], [6, 6], [98, 128], [14, 24], [95, 119], [96, 121], [4, 3], [6, 6], [95, 119], [95, 119], [97, 122], [20, 34], [102, 133], [6, 5], [95, 119], [6, 6], [95, 119], [103, 143], [95, 119], [5, 7], [14, 19], [21, 33], [4, 4], [95, 119], [98, 126], [18, 26], [95, 

In [14]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [16]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [29]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298], topo=[183, 24])


# TOPO_W_GSAGE

In [17]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [35]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.5786, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 02, Loss: 1.5134, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 03, Loss: 1.4844, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 04, Loss: 1.4571, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 05, Loss: 1.4178, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 06, Loss: 1.4043, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 07, Loss: 1.3632, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 08, Loss: 1.3067, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 09, Loss: 1.2916, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 10, Loss: 1.2885, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 11, Loss: 1.2774, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 12, Loss: 1.2191, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 13, Loss: 1.1846, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epo

Run: 01, Epoch: 122, Loss: 0.4897, Train: 89.66%, Valid: 79.66% Test: 81.08%
Run: 01, Epoch: 123, Loss: 0.4636, Train: 91.95%, Valid: 81.36% Test: 81.08%
Run: 01, Epoch: 124, Loss: 0.3969, Train: 93.10%, Valid: 84.75% Test: 83.78%
Run: 01, Epoch: 125, Loss: 0.4125, Train: 94.25%, Valid: 83.05% Test: 83.78%
Run: 01, Epoch: 126, Loss: 0.4677, Train: 94.25%, Valid: 79.66% Test: 75.68%
Run: 01, Epoch: 127, Loss: 0.4225, Train: 93.10%, Valid: 81.36% Test: 78.38%
Run: 01, Epoch: 128, Loss: 0.4509, Train: 83.91%, Valid: 77.97% Test: 70.27%
Run: 01, Epoch: 129, Loss: 0.5228, Train: 79.31%, Valid: 79.66% Test: 67.57%
Run: 01, Epoch: 130, Loss: 0.4637, Train: 86.21%, Valid: 81.36% Test: 78.38%
Run: 01, Epoch: 131, Loss: 0.3350, Train: 93.10%, Valid: 86.44% Test: 86.49%
Run: 01, Epoch: 132, Loss: 0.4113, Train: 94.25%, Valid: 88.14% Test: 86.49%
Run: 01, Epoch: 133, Loss: 0.4918, Train: 91.95%, Valid: 86.44% Test: 91.89%
Run: 01, Epoch: 134, Loss: 0.3962, Train: 91.95%, Valid: 86.44% Test: 86.49%

Run: 02, Epoch: 50, Loss: 0.7367, Train: 85.06%, Valid: 69.49% Test: 72.97%
Run: 02, Epoch: 51, Loss: 0.7030, Train: 85.06%, Valid: 72.88% Test: 70.27%
Run: 02, Epoch: 52, Loss: 0.7544, Train: 85.06%, Valid: 72.88% Test: 67.57%
Run: 02, Epoch: 53, Loss: 0.7478, Train: 85.06%, Valid: 72.88% Test: 72.97%
Run: 02, Epoch: 54, Loss: 0.7260, Train: 83.91%, Valid: 76.27% Test: 70.27%
Run: 02, Epoch: 55, Loss: 0.6727, Train: 83.91%, Valid: 76.27% Test: 67.57%
Run: 02, Epoch: 56, Loss: 0.7043, Train: 83.91%, Valid: 77.97% Test: 64.86%
Run: 02, Epoch: 57, Loss: 0.6774, Train: 85.06%, Valid: 77.97% Test: 64.86%
Run: 02, Epoch: 58, Loss: 0.6985, Train: 83.91%, Valid: 74.58% Test: 62.16%
Run: 02, Epoch: 59, Loss: 0.7443, Train: 83.91%, Valid: 74.58% Test: 59.46%
Run: 02, Epoch: 60, Loss: 0.6922, Train: 86.21%, Valid: 76.27% Test: 62.16%
Run: 02, Epoch: 61, Loss: 0.7447, Train: 86.21%, Valid: 76.27% Test: 67.57%
Run: 02, Epoch: 62, Loss: 0.7109, Train: 85.06%, Valid: 72.88% Test: 67.57%
Run: 02, Epo

Run: 02, Epoch: 181, Loss: 0.4106, Train: 94.25%, Valid: 86.44% Test: 72.97%
Run: 02, Epoch: 182, Loss: 0.3290, Train: 95.40%, Valid: 88.14% Test: 72.97%
Run: 02, Epoch: 183, Loss: 0.3712, Train: 96.55%, Valid: 84.75% Test: 72.97%
Run: 02, Epoch: 184, Loss: 0.2834, Train: 95.40%, Valid: 81.36% Test: 75.68%
Run: 02, Epoch: 185, Loss: 0.3195, Train: 94.25%, Valid: 81.36% Test: 75.68%
Run: 02, Epoch: 186, Loss: 0.3463, Train: 95.40%, Valid: 83.05% Test: 75.68%
Run: 02, Epoch: 187, Loss: 0.3102, Train: 96.55%, Valid: 81.36% Test: 75.68%
Run: 02, Epoch: 188, Loss: 0.3733, Train: 96.55%, Valid: 83.05% Test: 78.38%
Run: 02, Epoch: 189, Loss: 0.3015, Train: 96.55%, Valid: 84.75% Test: 75.68%
Run: 02, Epoch: 190, Loss: 0.2627, Train: 94.25%, Valid: 83.05% Test: 75.68%
Run: 02, Epoch: 191, Loss: 0.3758, Train: 95.40%, Valid: 84.75% Test: 75.68%
Run: 02, Epoch: 192, Loss: 0.3551, Train: 94.25%, Valid: 81.36% Test: 75.68%
Run: 02, Epoch: 193, Loss: 0.3714, Train: 95.40%, Valid: 81.36% Test: 75.68%

Run: 03, Epoch: 108, Loss: 0.4776, Train: 90.80%, Valid: 72.88% Test: 83.78%
Run: 03, Epoch: 109, Loss: 0.4420, Train: 90.80%, Valid: 74.58% Test: 78.38%
Run: 03, Epoch: 110, Loss: 0.4840, Train: 91.95%, Valid: 72.88% Test: 72.97%
Run: 03, Epoch: 111, Loss: 0.4025, Train: 93.10%, Valid: 74.58% Test: 70.27%
Run: 03, Epoch: 112, Loss: 0.4644, Train: 91.95%, Valid: 74.58% Test: 67.57%
Run: 03, Epoch: 113, Loss: 0.4091, Train: 90.80%, Valid: 76.27% Test: 72.97%
Run: 03, Epoch: 114, Loss: 0.4265, Train: 91.95%, Valid: 76.27% Test: 70.27%
Run: 03, Epoch: 115, Loss: 0.4223, Train: 91.95%, Valid: 76.27% Test: 72.97%
Run: 03, Epoch: 116, Loss: 0.5087, Train: 90.80%, Valid: 72.88% Test: 72.97%
Run: 03, Epoch: 117, Loss: 0.4657, Train: 86.21%, Valid: 66.10% Test: 72.97%
Run: 03, Epoch: 118, Loss: 0.4835, Train: 83.91%, Valid: 67.80% Test: 75.68%
Run: 03, Epoch: 119, Loss: 0.4447, Train: 83.91%, Valid: 66.10% Test: 72.97%
Run: 03, Epoch: 120, Loss: 0.4649, Train: 86.21%, Valid: 64.41% Test: 64.86%

...

Run: 07, Epoch: 197, Loss: 0.3062, Train: 98.85%, Valid: 81.36% Test: 89.19%
Run: 07, Epoch: 198, Loss: 0.4200, Train: 97.70%, Valid: 76.27% Test: 89.19%
Run: 07, Epoch: 199, Loss: 0.2592, Train: 97.70%, Valid: 77.97% Test: 89.19%
Run: 07, Epoch: 200, Loss: 0.2799, Train: 97.70%, Valid: 74.58% Test: 86.49%
Run 07:
Highest Train: 98.85
Highest Valid: 84.75
  Final Train: 97.70
   Final Test: 89.19
Run: 08, Epoch: 01, Loss: 1.5544, Train: 9.20%, Valid: 11.86% Test: 13.51%
Run: 08, Epoch: 02, Loss: 1.5089, Train: 43.68%, Valid: 49.15% Test: 43.24%
Run: 08, Epoch: 03, Loss: 1.4700, Train: 45.98%, Valid: 49.15% Test: 45.95%
Run: 08, Epoch: 04, Loss: 1.4245, Train: 42.53%, Valid: 47.46% Test: 45.95%
Run: 08, Epoch: 05, Loss: 1.4144, Train: 42.53%, Valid: 47.46% Test: 45.95%
Run: 08, Epoch: 06, Loss: 1.3496, Train: 42.53%, Valid: 47.46% Test: 45.95%
Run: 08, Epoch: 07, Loss: 1.3037, Train: 42.53%, Valid: 47.46% Test: 45.95%
Run: 08, Epoch: 08, Loss: 1.2858, Train: 42.53%, Valid: 47.46% Test: 

Run: 08, Epoch: 124, Loss: 0.4206, Train: 93.10%, Valid: 84.75% Test: 67.57%
Run: 08, Epoch: 125, Loss: 0.3562, Train: 95.40%, Valid: 79.66% Test: 75.68%
Run: 08, Epoch: 126, Loss: 0.4277, Train: 93.10%, Valid: 79.66% Test: 72.97%
Run: 08, Epoch: 127, Loss: 0.3526, Train: 89.66%, Valid: 77.97% Test: 70.27%
Run: 08, Epoch: 128, Loss: 0.3827, Train: 91.95%, Valid: 77.97% Test: 70.27%
Run: 08, Epoch: 129, Loss: 0.4743, Train: 91.95%, Valid: 81.36% Test: 72.97%
Run: 08, Epoch: 130, Loss: 0.4137, Train: 93.10%, Valid: 84.75% Test: 70.27%
Run: 08, Epoch: 131, Loss: 0.4348, Train: 95.40%, Valid: 83.05% Test: 72.97%
Run: 08, Epoch: 132, Loss: 0.4557, Train: 96.55%, Valid: 84.75% Test: 78.38%
Run: 08, Epoch: 133, Loss: 0.3999, Train: 96.55%, Valid: 84.75% Test: 78.38%
Run: 08, Epoch: 134, Loss: 0.3994, Train: 96.55%, Valid: 84.75% Test: 78.38%
Run: 08, Epoch: 135, Loss: 0.4162, Train: 94.25%, Valid: 86.44% Test: 83.78%
Run: 08, Epoch: 136, Loss: 0.3916, Train: 91.95%, Valid: 84.75% Test: 75.68%

Run: 09, Epoch: 51, Loss: 0.7400, Train: 88.51%, Valid: 83.05% Test: 81.08%
Run: 09, Epoch: 52, Loss: 0.6916, Train: 83.91%, Valid: 83.05% Test: 78.38%
Run: 09, Epoch: 53, Loss: 0.6888, Train: 77.01%, Valid: 83.05% Test: 75.68%
Run: 09, Epoch: 54, Loss: 0.7038, Train: 77.01%, Valid: 83.05% Test: 78.38%
Run: 09, Epoch: 55, Loss: 0.7056, Train: 80.46%, Valid: 81.36% Test: 75.68%
Run: 09, Epoch: 56, Loss: 0.6679, Train: 79.31%, Valid: 81.36% Test: 75.68%
Run: 09, Epoch: 57, Loss: 0.6719, Train: 75.86%, Valid: 81.36% Test: 75.68%
Run: 09, Epoch: 58, Loss: 0.6876, Train: 74.71%, Valid: 81.36% Test: 75.68%
Run: 09, Epoch: 59, Loss: 0.7194, Train: 79.31%, Valid: 81.36% Test: 78.38%
Run: 09, Epoch: 60, Loss: 0.6408, Train: 81.61%, Valid: 81.36% Test: 78.38%
Run: 09, Epoch: 61, Loss: 0.6623, Train: 87.36%, Valid: 81.36% Test: 72.97%
Run: 09, Epoch: 62, Loss: 0.6761, Train: 87.36%, Valid: 83.05% Test: 75.68%
Run: 09, Epoch: 63, Loss: 0.6444, Train: 86.21%, Valid: 79.66% Test: 78.38%
Run: 09, Epo

Run: 09, Epoch: 179, Loss: 0.4470, Train: 95.40%, Valid: 91.53% Test: 86.49%
Run: 09, Epoch: 180, Loss: 0.3646, Train: 96.55%, Valid: 88.14% Test: 83.78%
Run: 09, Epoch: 181, Loss: 0.3639, Train: 94.25%, Valid: 86.44% Test: 81.08%
Run: 09, Epoch: 182, Loss: 0.4182, Train: 94.25%, Valid: 88.14% Test: 81.08%
Run: 09, Epoch: 183, Loss: 0.3895, Train: 95.40%, Valid: 89.83% Test: 83.78%
Run: 09, Epoch: 184, Loss: 0.4141, Train: 95.40%, Valid: 89.83% Test: 86.49%
Run: 09, Epoch: 185, Loss: 0.3797, Train: 95.40%, Valid: 88.14% Test: 83.78%
Run: 09, Epoch: 186, Loss: 0.3862, Train: 96.55%, Valid: 91.53% Test: 83.78%
Run: 09, Epoch: 187, Loss: 0.4293, Train: 95.40%, Valid: 91.53% Test: 83.78%
Run: 09, Epoch: 188, Loss: 0.3776, Train: 95.40%, Valid: 93.22% Test: 83.78%
Run: 09, Epoch: 189, Loss: 0.3731, Train: 95.40%, Valid: 93.22% Test: 86.49%
Run: 09, Epoch: 190, Loss: 0.4384, Train: 95.40%, Valid: 93.22% Test: 86.49%
Run: 09, Epoch: 191, Loss: 0.3835, Train: 95.40%, Valid: 93.22% Test: 86.49%

Run: 10, Epoch: 107, Loss: 0.5445, Train: 87.36%, Valid: 86.44% Test: 75.68%
Run: 10, Epoch: 108, Loss: 0.5033, Train: 87.36%, Valid: 83.05% Test: 72.97%
Run: 10, Epoch: 109, Loss: 0.4976, Train: 88.51%, Valid: 83.05% Test: 75.68%
Run: 10, Epoch: 110, Loss: 0.4474, Train: 90.80%, Valid: 81.36% Test: 75.68%
Run: 10, Epoch: 111, Loss: 0.5124, Train: 88.51%, Valid: 81.36% Test: 75.68%
Run: 10, Epoch: 112, Loss: 0.4878, Train: 89.66%, Valid: 83.05% Test: 78.38%
Run: 10, Epoch: 113, Loss: 0.5224, Train: 90.80%, Valid: 86.44% Test: 81.08%
Run: 10, Epoch: 114, Loss: 0.4633, Train: 91.95%, Valid: 86.44% Test: 83.78%
Run: 10, Epoch: 115, Loss: 0.5296, Train: 94.25%, Valid: 84.75% Test: 81.08%
Run: 10, Epoch: 116, Loss: 0.5199, Train: 91.95%, Valid: 81.36% Test: 81.08%
Run: 10, Epoch: 117, Loss: 0.5578, Train: 87.36%, Valid: 81.36% Test: 75.68%
Run: 10, Epoch: 118, Loss: 0.4604, Train: 91.95%, Valid: 84.75% Test: 78.38%
Run: 10, Epoch: 119, Loss: 0.4629, Train: 91.95%, Valid: 91.53% Test: 83.78%

# TOPO-GSAGE

In [18]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298], topo=[183, 24])


In [20]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, ' f'Train: {100 * train_acc:.2f}%, '
                 #     f'Valid: {100 * valid_acc:.2f}% '
                  #    f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run 01:
Highest Train: 100.00
Highest Valid: 72.88
  Final Train: 97.70
   Final Test: 70.27
Run 02:
Highest Train: 100.00
Highest Valid: 64.41
  Final Train: 77.01
   Final Test: 56.76
Run 03:
Highest Train: 100.00
Highest Valid: 71.19
  Final Train: 96.55
   Final Test: 59.46
Run 04:
Highest Train: 100.00
Highest Valid: 59.32
  Final Train: 93.10
   Final Test: 62.16
Run 05:
Highest Train: 100.00
Highest Valid: 64.41
  Final Train: 90.80
   Final Test: 56.76
Run 06:
Highest Train: 100.00
Highest Valid: 69.49
  Final Train: 100.00
   Final Test: 62.16
Run 07:
Highest Train: 100.00
Highest Valid: 67.80
  Final Train: 100.00
   Final Test: 72.97
Run 08:
Highest Train: 100.00
Highest Valid: 74.58
  Final Train: 97.70
   Final Test: 59.46
Run 09:
Highest Train: 100.00
Highest Valid: 69.49
  Final Train: 80.46
   Final Test: 62.16
Run 10:
Highest Train: 100.00
Highest Valid: 77.97
  Final Train: 90.80
   Final Test: 54.05
All runs:
Highest Train: 100.00 ± 0.00
Highest Valid: 69.15 ± 5.47
 